In [49]:
import pandas as pd

In [50]:
df = pd.read_csv('srpski.csv', sep='\t')
df = df.drop(columns=['Rbr', 'SR', 'sr/sr', 'Naslov', 'Jezik'])

ParserError: Error tokenizing data. C error: Expected 7 fields in line 140, saw 13


In [ ]:
from nltk import FreqDist

In [ ]:
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = df[column].str.lower()

In [ ]:
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
nltk.download('punkt')


[nltk_data] Error loading punkt: <urlopen error [Errno 8] nodename nor
[nltk_data]     servname provided, or not known>


False

In [ ]:
custom_tokenizer = RegexpTokenizer(r'\w+[\'\’]\w+|\w+')
df['Tokeni'] = df['Tekst'].apply(custom_tokenizer.tokenize)
df['Tokeni']


0      [padajte, braćo, plin'te, u, krvi, ostav'te, s...
1      [mudraci, su, prinosili, dara, smirnu, zlato, ...
2      [mnogi, me, je, dosad, zapitkiv’o, sa, čega, s...
3      [ja, sam, stena, o, koju, se, zloba, mori, sve...
4      [kroz, ponoć, nemu, i, gusto, granje, vidi, se...
                             ...                        
135    [već, sanduk, zatvaraju, čvrsto, da, večno, bu...
136    [ne, pitaj, me, o, bosforu, više, ja, ti, ne, ...
137    [ko, sam, šta, sam, ja, sam, samo, sanjar, čij...
138    [prastara, ispali, zubi, na, rogu, svitkovi, l...
139    [zar, sam, ja, kriv, što, mi, život, nije, mio...
Name: Tokeni, Length: 140, dtype: object

In [ ]:
all_words = [word for words in df['Tokeni'] for word in words]
fdist = FreqDist(all_words)


In [ ]:
most_common_words = fdist.most_common(70)
stopwords = [word for word,count in most_common_words]
punctuation = ['.', ',', '``', "'", "''",'...','—',"-",';', ':' ]

In [ ]:
def remove_stopwords_and_punctuation(tokens): 
    filtered_tokens = [word for word in tokens if word not in punctuation ]
    return filtered_tokens

In [ ]:
df['Tokeni'] = df['Tokeni'].apply(remove_stopwords_and_punctuation)
df['Tokeni']

0      [padajte, braćo, plin'te, u, krvi, ostav'te, s...
1      [mudraci, su, prinosili, dara, smirnu, zlato, ...
2      [mnogi, me, je, dosad, zapitkiv’o, sa, čega, s...
3      [ja, sam, stena, o, koju, se, zloba, mori, sve...
4      [kroz, ponoć, nemu, i, gusto, granje, vidi, se...
                             ...                        
135    [već, sanduk, zatvaraju, čvrsto, da, večno, bu...
136    [ne, pitaj, me, o, bosforu, više, ja, ti, ne, ...
137    [ko, sam, šta, sam, ja, sam, samo, sanjar, čij...
138    [prastara, ispali, zubi, na, rogu, svitkovi, l...
139    [zar, sam, ja, kriv, što, mi, život, nije, mio...
Name: Tokeni, Length: 140, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
author_texts = df['Tokeni'].apply(' '.join)
vectorizer = TfidfVectorizer()
tf_matrix = vectorizer.fit_transform(author_texts)
tf_df = pd.DataFrame(tf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tf_df

,1938,39,afrodita,age,agnus,ah,aj,ajd,ajde,ajdemo,...,žudite,žudnih,žudno,žulji,žut,žute,žuti,žutih,žutilo,žućkastih
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tf_matrix, df['Autor'], test_size=0.2, random_state=42, stratify=df['Autor'])

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier


In [ ]:
classifierMB = MultinomialNB()
#df_random = pd.DataFrame(X_train.toarray(), columns= vectorizer.get_feature_names_out())
classifierMB.fit(X_train,y_train)
y_pred = classifierMB.predict(X_test)
y_pred_train = classifierMB.predict(X_train)
report = accuracy_score(y_test, y_pred)
report1 = accuracy_score(y_train, y_pred_train)
report


0.2857142857142857

In [ ]:
model = GradientBoostingClassifier(learning_rate=0.1,max_depth=5,min_samples_split=2, n_estimators=300)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
report = accuracy_score(y_test, y_pred)
report

0.10714285714285714

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model1 = RandomForestClassifier(max_depth=5,min_samples_split=15, n_estimators=300)
model1.fit(X_train, y_train)
y_pred = model1.predict(X_test)
report = accuracy_score(y_test, y_pred)
report

0.17857142857142858

In [ ]:
# Create an SVM model
from sklearn.svm import SVC
svm_model = SVC(kernel='linear', C=1.0)  # You can adjust the kernel and other hyperparameters

# Train the model
svm_model.fit(X_train, y_train)


SVC(kernel='linear')

In [ ]:
# Make predictions on the test set
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy


0.32142857142857145